# Discretized Differential Equations

Explain methods of finite differences and elements (Galerkin methods).

## Linear System via Discretization

In [1]:
"""
Specify a discretized differential equation as a linear operator and solve it.
"""

'\nSpecify a discretized differential equation as a linear operator and solve it.\n'

## Propagation of Uncertainty

In [2]:
"""
Solve multiple systems with previous posterior as a prior.
"""

'\nSolve multiple systems with previous posterior as a prior.\n'